In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#===================== Parameter settings =========================
#for training
algorithms = {
    "LS2IC": "../results/ls2ic",
    "MADQN": "../results/ps_dqn",
    "MADQN-PA": "../results/ps_dqn_a",
    "MF-Q": "../results/meanfield"
}
num_agents = 992 #32*31, geant topolopy is 506 = 23*22

# for testing
algorithm_dirs = {
    "LS2IC": "../results/ls2ic",
    "MADQN": "../results/ps_dqn",
    "MADQN-PA": "../results/ps_dqn_a",
    "MF-Q": "../results/meanfield",
    "DRSIR": "../results/drsir",
    "OSPF": "../results/ospf"
}

# testing tm id
tm_files = ["06_eval_metrics.csv", "41_eval_metrics.csv", "73_eval_metrics.csv", "108_eval_metrics.csv", "141_eval_metrics.csv"]

In [ ]:
def get_smooth(values, smoothing_factor):
    smoothed = []
    for i, val in enumerate(values):
        if i == 0:
            smoothed.append(val)
        else:
            smoothed.append(val * (1 - smoothing_factor) + smoothed[i - 1] * smoothing_factor)
    return smoothed

def load_rewards_from_file(file_path, divisor=992):
    if not os.path.isfile(file_path):
        raise FileNotFoundError(f"can't found: {file_path}")

    with open(file_path, 'r') as f:
        lines = f.read().splitlines()

    raw_tokens = ",".join(lines).split(',')
    float_rewards = []
    for token in raw_tokens[1:-1]:
        try:
            val = float(token) / divisor
            float_rewards.append(val)
        except ValueError:
            continue
    return float_rewards

def plot_algorithms(algorithms, smoothing_factor=0.95, divisor=992, ylabel="Average reward", output_file="training_reward.svg"):
    plt.rcParams.update({'font.size': 14})
    plt.figure(figsize=(10, 6))
    plt.xlabel('Step', fontsize=16)
    plt.ylabel(ylabel, fontsize=16)

    colors = plt.cm.tab10.colors 
    for idx, (label, dir_path) in enumerate(algorithms.items()):
        file_path = os.path.join(dir_path, "output.txt")
        rewards = load_rewards_from_file(file_path, divisor=divisor)
        if not rewards:
            continue
        smoothed = get_smooth(rewards, smoothing_factor)
        steps = np.arange(len(smoothed))
        color = colors[idx % len(colors)]
        plt.plot(steps, smoothed, alpha=0.5, color=color, label=label)

    plt.legend(fontsize=16)
    plt.ylim(bottom=30)
    plt.tight_layout()
    plt.savefig(output_file, format='svg', bbox_inches='tight')
    plt.show()

if __name__ == "__main__":
    plot_algorithms(
        algorithms=algorithms,
        smoothing_factor=0.95,
        divisor=num_agents,
        ylabel="Average reward",
        output_file="Training_Reward.svg"
    )


In [ ]:
metrics_data = {alg: {} for alg in algorithm_dirs}

# Read each algorithm's CSV file and calculate the average of each column
for file in tm_files:
    tm_id = file.split("_")[0]  # Get TM ID
    
    for alg, dir_path in algorithm_dirs.items():
        file_path = os.path.join(dir_path, file)
        try:
            df = pd.read_csv(file_path)
            metrics_data[alg][tm_id] = {
                "avg_delay": df["avg_delay"].mean(),
                "avg_packet_loss": df["avg_packet_loss"].mean(),
                "avg_throughput": df["avg_throughput"].mean(),
                "max_link_utilization": df["max_link_utilization"].mean()
            }
        except Exception as e:
            print(f"Error reading {file} ({alg}):", e)

# Print results for verification
for alg, data in metrics_data.items():
    print(f"Algorithm {alg} metrics:")
    print(data)

def plot_all_metrics(metrics_data):
    """
    Plot all four metrics for each algorithm, adaptable to different numbers of algorithms.
    """
    metrics = ["avg_delay", "avg_packet_loss", "avg_throughput", "max_link_utilization"]
    titles = ["(a) Average Link Delay", "(b) Average Link Packet Loss", "(c) Average Link Throughput", "(d) Maximum Link Utilization"]
    ylabels = ["Delay (ms)", "Packet Loss (%)", "Throughput (Mb/s)", "Utilization (%)"]
    
    tm_ids = sorted(next(iter(metrics_data.values())).keys(), key=lambda x: int(x))
    x = np.arange(len(tm_ids))  # X-axis ticks
    width = 0.8 / len(metrics_data)  # Dynamically adjust bar width
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    axes = axes.flatten()
    
    colors = ["#E41A1C", "#377EB8", "#4DAF4A", "#FF7F00", "#984EA3", "#A65628"]
    
    for i, metric in enumerate(metrics):
        ax = axes[i]
        for j, (alg, data) in enumerate(metrics_data.items()):
            values = [data[tm][metric] for tm in tm_ids]
            ax.bar(x + (j - len(metrics_data)/2 + 0.5) * width, values, width, label=alg, color=colors[j % len(colors)])
        
        ax.set_xlabel(titles[i], fontsize=16)
        ax.set_ylabel(ylabels[i], fontsize=16)
        ax.set_xticks(x)
        ax.set_xticklabels(tm_ids, fontsize=14)
        ax.tick_params(axis='y', labelsize=14)
    
    fig.legend(metrics_data.keys(), loc='upper center', bbox_to_anchor=(0.5, 1.06), ncol=len(metrics_data.keys()), fontsize=14)
    fig.tight_layout(rect=[0, 0, 1, 1])
    plt.savefig("Figure 18. Testing result in 32-node 144TM.svg", format='svg', bbox_inches='tight')
    plt.show()

def compute_relative_improvements_no_avg(metrics_data):
    """
    Compute relative improvement tables (5×5 for alg×alg) for each metric & TM
    without averaging over TMs.

    Return structure: improvements_tables[metric][tm] = pd.DataFrame(alg x alg)
    Each cell stores the improvement percentage of A compared to B.
    """
    # Define which metrics are "lower is better" or "higher is better"
    metric_directions = {
        'avg_delay': 'lower',
        'avg_packet_loss': 'lower',
        'avg_throughput': 'higher',
        'max_link_utilization': 'lower'
    }

    metrics = list(metric_directions.keys())        # Four metrics
    alg_list = list(metrics_data.keys())            # Five algorithms
    
    # Get TM IDs from the first algorithm (e.g., 06, 41, 73, 108, 141)
    example_alg = alg_list[0]
    tm_ids = sorted(metrics_data[example_alg].keys(), key=lambda x: int(x))

    # Final structure to store all tables: {metric -> {tm -> df(5×5)}}
    improvement_tables = {}

    for metric in metrics:
        direction = metric_directions[metric]
        improvement_tables[metric] = {}  # Per-TM tables for this metric

        for tm in tm_ids:
            # Create a 5×5 table for this metric & TM (row: algA, col: algB)
            df_improvement = pd.DataFrame(index=alg_list, columns=alg_list, dtype=float)

            for algA in alg_list:
                for algB in alg_list:
                    if algA == algB:
                        df_improvement.loc[algA, algB] = np.nan
                        continue
                    
                    valA = metrics_data[algA][tm][metric]
                    valB = metrics_data[algB][tm][metric]

                    # Avoid division by zero
                    if valB == 0:
                        improvement = np.nan
                    else:
                        # lower: (valB - valA)/valB * 100  (lower A is better)
                        # higher: (valA - valB)/valB * 100 (higher A is better)
                        if direction == 'lower':
                            improvement = (valB - valA) / valB * 100.0
                        else:
                            improvement = (valA - valB) / valB * 100.0

                    df_improvement.loc[algA, algB] = improvement
            
            # Store the table
            improvement_tables[metric][tm] = df_improvement

    return improvement_tables

def print_improvement_tables_no_avg(improvement_tables):
    """
    Print the non-averaged relative improvement results (4×{TM->5×5}) all at once.
    """
    for metric, tm_dict in improvement_tables.items():
        print(f"=== {metric} ===")
        for tm, df in tm_dict.items():
            print(f"--- TM = {tm} ---")
            print(df.to_string(float_format=lambda x: f"{x:6.2f}%"))
            print()

if __name__ == "__main__":
    win_tables = compute_relative_improvements_no_avg(metrics_data)
    print_improvement_tables_no_avg(win_tables)
    plot_all_metrics(metrics_data)
